<a href="https://colab.research.google.com/github/gunisha30/codes/blob/master/Copy_of_audio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install praat-parselmouth

In [0]:
#convert mp4 to wav 
import sys
import os
from moviepy.editor import *

# name is the path, if its in the root directory then its just the video name
def convert_to_audio(name):
  video = VideoFileClip(name)
  audio = video.audio
  audio.write_audiofile('Q6converted.wav')

#folder renaming: first-Nishkarsh second-Yash third-Vishesh

os.chdir('/content/drive/My Drive/awarathon/third')
name="Q6.mp4"
convert_to_audio(name)

In [44]:
import parselmouth
from parselmouth.praat import call, run_file
import inspect as i
import sys
import scipy.io.wavfile
import math
import moviepy.editor as mp


def mysppaus(m,p,l):              #pause rate and articulation rate
    sound=p+"/"+m+".wav"          #entire path till audio file
    sourcerun="/content/drive/My Drive/awarathon/myspsolution.praat"                  #path till mysolution.praat
    path=p                        #path till the folder which has audio file
    try:
        objects= run_file(sourcerun, -20, 2, 1, "yes",sound,path, 80, 400, 0.01, capture_output=True) 
        z1=str(objects[1])
        z2=z1.strip().split()
        print ("articulation rate=",z2[3])
        p=int(z2[1])                          #pause length in secs
        p=p/60                                #pause length in mins
        a=p/l                                 #ratio of pause length and total length
        print("rate of pause=",a)
        print ("pause percentage=",a*100)
    except:
        print ("Try again the sound of the audio was not clear")
    return;


audioname="Q6converted"
path1="/content/drive/My Drive/awarathon/third"

#length of video files 

clip = mp.VideoFileClip('/content/drive/My Drive/awarathon/third/Q6.mp4')
duration = math.floor(clip.duration)
image_time_interval = 2
duration=duration/60
print("length=",duration)

mysppaus(audioname,path1,duration)

length= 0.15
articulation rate= 4
rate of pause= 0.0
pause percentage= 0.0


In [0]:
def audiolength(audio,path): #length of audio in mins
    rate, arr = scipy.io.wavfile.read(path+"/"+audio+".wav")
    length=arr.size/rate
    length=length/60
    return(length)


In [0]:
from google.colab import drive
drive.mount('/content/drive')